-- ANTES DE INICIAR..   CORRER ModificacionEsquema.sql que se encarga de 
-- hacer las transformaciones de los datos, asumiendo que se acaba de 
-- cargar los excel de datos.


## **UAOPiloto**
Detección de estudiantes que van a desertar en 2 períodos.

TABLAS BASE:  DATOS

TABLAS MYSQL:
1. estudiantes_limpio:  es la tabla ESTUDIANTE pero ahora si limpia
2. periodosRegulares:   es la tabla de periodos y ya viene con un orden que me permite identificar " los siguientes dos periodos"
3.  

# **1.  Conexión a la base de datos**
Nos conectamos a la base de datos MYSQL

In [7]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)

# **2.  crear DATOS**

Se crea DATOS a partir de DISTINCT EstudianteAsignatura(id_estudiante,periodo)

In [8]:
query = """
    SELECT distinct id_estudiante, periodo
    FROM estudianteAsignatura
    ORDER BY id_estudiante, periodo
"""
DATOS = pd.read_sql(query, con=motor)

# NOTA:  falta manejar el caso de cursos de verano...  cuándo marcar al estudainte como desertor?  
# ERIK ya tiene la pregunta 20231009. R/  indica ERIK solo considerar períodos regulares

### **2.1 adicionamos el primer periodo que hizo matricula**
De la tabla EstudiantesMatriculas adicionamos el primer periodo en que el estudiante hizo su primer matricula
EstudiantesMatriculas(id_estudiante, MIN(periodo))

In [9]:
# adicionamos el primer periodo en que ese estudiante hizo una matricula
query = """
    SELECT  id_estudiante, min(periodo) periodo_inicial
    FROM estudianteAsignatura
    WHERE periodo like '%1' or periodo like '%3'
    GROUP BY id_estudiante
"""
temp = pd.read_sql(query, con=motor)
# hacemos join entre DATOS y PRIMER_PERIODO para adicionar el periodo_inicial
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **2.2 Adicionamos fecha_grado, fecha_nacimiento, genero, colegio, ciudad_colegio**
De la tabla Estudiantes_Limpia saco esta información por estudiante y la adicionamos a DATOS

In [10]:
# ahora adicionamos el periodo de grado Y TODAS LAS COLUMNAS REQUERIDAS DESDE ESTUDIANTES_LIMPIO 
# pues ya hice todo el trabajo
# de limpieza.

query = """
    SELECT  id_estudiante, fecha_grado , fecha_nacimiento, genero, colegio, ciudad_colegio
    FROM estudiantes_limpio
"""
temp = pd.read_sql(query, con=motor)

# hacemos join con DATOS para adicionar la fecha de grado
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

### **2.3 adicionamos el número del periodo (periodo_orden)**

en la tabla PERIODOS está el orden de cada periodo.  vamos a adicionarlos.


In [11]:
query = """
    SELECT  pr.periodo, pr.orden periodo_orden, pr.fecha_inicio periodo_fi, pr.fecha_final periodo_ff
    from periodosRegulares pr
"""
temp = pd.read_sql(query, con=motor)

DATOS = DATOS.merge(temp, on=['periodo'], how='left')


SALVEMOS DATOS para que no nos toque procesar lo anterior

In [13]:
DATOS.to_excel('DATOS.xlsx')

### **2.3 LABEL:  calculamos si dada una fila en d de DATOS, los siguientes 2 periodos no matriculó (ni se graduó)**

Ahora procedemos a generar LABEL: 1- el estudiante desertó.  0- el estudiante no desertó

hacemos un FOR en DATOS
por cada d (en el periodo i)
   miramos si i e i+1 tiene matricula: 
   si si:   lo marcamos como 0 -  
   si no:  miramos si se graduó en i, i+1 o i+2.  
           si si:  marcamos como 0
           si no:  marcamos como 1
   

In [14]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)
DATOS = pd.read_excel('DATOS.xlsx')

In [15]:
# el máximo periodo_orden es 21!! OJO, ESTO HAY QUE CALCULARLO

from datetime import datetime, timedelta

DATOS['LABEL'] = -1   # asumimos que no se puede calcular

for indice, d in DATOS.iterrows():
    
    id_estudiante = d['id_estudiante']
    periodo_orden = d['periodo_orden']

    # print(id_estudiante, periodo_orden,'fecha_grado:', d['fecha_grado'], 'periodo_fi:', d['periodo_fi'], d['periodo_fi'] + timedelta(days=365))


    if periodo_orden >= 20:  # si es 20 o 21 no podemos aún calcular si si o si no
        d['LABEL'] = -1
        # print('     no se sabe')
    else:
        existe_periodo_1 =  DATOS[(DATOS['id_estudiante'] == id_estudiante) & (DATOS['periodo_orden'] == periodo_orden +1)]
        existe_periodo_2 =  DATOS[(DATOS['id_estudiante'] == id_estudiante) & (DATOS['periodo_orden'] == periodo_orden +2)]

        # si matriculo periodo 1 o periodo 2
        if len(existe_periodo_1)>0 or len(existe_periodo_2)>0:
            
            # como matriculó al menos uno de los dos siguientes periodos, no es desertor.
            DATOS.at[indice, 'LABEL'] = 0  

        else: 

            # si entra aquí es porque no matriculo ninguno de los dos siguientes períodos

            # si no tiene fecha de grado, es desertor.
            if pd.isna(d['fecha_grado']):
                DATOS.at[indice, 'LABEL'] = 1
            else:  
                
                # si sí tiene fecha de grado, pero es posterior a 1 año, es desertor.
                if d['fecha_grado'] > d['periodo_fi'] + timedelta(days=365):  
                    DATOS.at[indice, 'LABEL'] = 1
                else:
                    # si la fecha de grado ocurre antes de un año, NO es desertor.
                    DATOS.at[indice, 'LABEL'] = 0


In [16]:
DATOS.columns

Index(['Unnamed: 0', 'id_estudiante', 'periodo', 'periodo_inicial',
       'fecha_grado', 'fecha_nacimiento', 'genero', 'colegio',
       'ciudad_colegio', 'periodo_orden', 'periodo_fi', 'periodo_ff', 'LABEL'],
      dtype='object')

# **3.  Adicionar variables de interés**

El excel HIPOTESIS vs VARIABLES va explicando qué variables se van a ir adicionando

### **3.1 Resultados por áreas icfes**
Se adiciona el resultado en cada área que obtuvo el estudiante, incluyendo el resultado en ICFES_ANTIGUO
Si hay nulo se coloca -100

In [17]:
query = """
    SELECT  id_estudiante, IFNULL(icfes_antiguo,-100) icfes_antiguo, IFNULL(biologia,-100) biologia, IFNULL(matematica,-100) matematica, 
			IFNULL(filosofia,-100) filosofia, 
            IFNULL(fisica,-100) fisica, IFNULL(historia,-100) historia, IFNULL(quimica,-100) quimica, IFNULL(lenguaje,-100) lenguaje, 
            IFNULL(geografia,-100) geografia, IFNULL(idioma,-100) idioma, IFNULL(interdisciplinar,-100) interdisciplinar
    FROM icfes;
"""
temp = pd.read_sql(query, con=motor)
# hacemos join (temporal) contra datos y vamos a mirar cómo podemos agrupar cada variable frente a LABEL
DATOS = DATOS.merge(temp, on=['id_estudiante'], how='left')

In [18]:
DATOS = DATOS.fillna(0)
DATOS

,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,periodo_orden,...,biologia,matematica,filosofia,fisica,historia,quimica,lenguaje,geografia,idioma,interdisciplinar
0,0,2136680,201303,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2136680,201401,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2136680,201403,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2136680,201501,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2136680,201503,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,1338,2231877,202303,202301,0,2006-02-05 00:00:00,MAS,INSTITUCION EDUCATIVA PRIMAVERA,BOLIVAR,21,...,58.0,53.0,53.0,50.0,50.0,58.0,50.0,50.0,57.0,-100.0
1339,1339,2232058,202301,202301,0,2005-12-01 00:00:00,MAS,OTRO,CALI,20,...,56.0,74.0,80.0,74.0,74.0,56.0,80.0,74.0,74.0,-100.0
1340,1340,2232058,202303,202301,0,2005-12-01 00:00:00,MAS,OTRO,CALI,21,...,56.0,74.0,80.0,74.0,74.0,56.0,80.0,74.0,74.0,-100.0
1341,1341,2232126,202301,202301,0,2005-01-10 00:00:00,MAS,COLEGIO MAYOR SAN FRANCISCO DE ASIS,CALI,20,...,-100.0,59.0,57.0,59.0,51.0,55.0,57.0,55.0,55.0,-100.0


### **3.2 Distribución notas obtenidas semestres anteriores**

Para poder obtener esta información correctamente es conveniente NORMALIZAR la tabla EstudianteAsignatura
en 3 tablas:

EstudianteAsignatura:  info del estudiante y la asignatura que matrículo. PK= (periodo, id_estudiante, cod_asig) 
EA_grupo:  que tiene como pk (periodo, id_estudiante, cod_asig, grupo)
EA_docentes:  que tiene como pk (periodo, id_estudiante, cod_asig, grupo, cedula)

Y también se creó la columna nivel_nota en estudianteasignatura que nos dice cuantas de ese tipo sacó. 
if definitiva null ->  nivel_nota = -1
if definitiva menor a 3.0 -> nivel_nota = 1
if definitiva entre 3 y menor a 3.8 -> nivel_nota = 2
if definitiva mayor o igual a 3.8 -> nivel_nota = 3

por cada fila d en DATOS:
   obtener la cantidad de notas nulas, bajas, medias, altas que obtuvo en períodos anteriores

In [19]:
query = """
    SELECT  ae.id_estudiante, ae.periodo, pr.orden periodo_orden, ae.nivel_nota, count(*) cuantas
    FROM estudianteasignatura ae, periodosregulares pr
    WHERE pr.periodo = ae.periodo
    group by ae.id_estudiante, ae.periodo, pr.orden, ae.nivel_nota
"""
temp = pd.read_sql(query, con=motor)

# temp = temp.pivot(index=["id_estudiante", "orden"], columns="nivel_nota", values="cuantas").reset_index()
# temp = temp.fillna(0)
print(temp.columns)


Index(['id_estudiante', 'periodo', 'periodo_orden', 'nivel_nota', 'cuantas'], dtype='object')


In [50]:
DATOS.columns

Index(['Unnamed: 0', 'id_estudiante', 'periodo', 'periodo_inicial',
       'fecha_grado', 'fecha_nacimiento', 'genero', 'colegio',
       'ciudad_colegio', 'periodo_orden', 'periodo_fi', 'periodo_ff', 'LABEL',
       'icfes_antiguo', 'biologia', 'matematica', 'filosofia', 'fisica',
       'historia', 'quimica', 'lenguaje', 'geografia', 'idioma',
       'interdisciplinar'],
      dtype='object')

In [22]:
# seleccionarmos solo id_estudiante, periodo y periodo_orden de DATOS
temp2 = DATOS[['id_estudiante','periodo','periodo_orden']]

# hacemos left join con temp
temp2 = temp2.merge(temp, on=['id_estudiante'], how='left')

# dejamos solo las filas donde periodo_orden_x > periodo_orden_y pues queremos mirar al periodo cuantas
# materias tuvo sin nota, cuantas tuvo con nivel 1, cuantas con nivel 2 y cuantas con nivel 3 de periodos
# anteriores
temp2 = temp2[temp2['periodo_orden_x'] > temp2['periodo_orden_y']]

# ahora agrupamos para saber el total de materias con nulo, 1,2 o 4 de periodos anteriores.
temp2 = temp2.groupby(['id_estudiante','periodo_x','nivel_nota'])['cuantas'].sum().reset_index()

# pivoteamos
temp2 = temp2.pivot(index=['id_estudiante','periodo_x'], columns='nivel_nota', values='cuantas').reset_index()

# renombramos y llenamos con ceros
temp2 = temp2.rename(columns={'periodo_x':'periodo', 
                              '-1':'per_ant_nivel_menos_1',
                              '1':'per_ant_nivel_1',
                              '2':'per_ant_nivel_2',
                              '3':'per_ant_nivel_3'})
temp2 = temp2.fillna(0)
temp2

nivel_nota,id_estudiante,periodo,per_ant_nivel_menos_1,per_ant_nivel_1,per_ant_nivel_2,per_ant_nivel_3
0,2136680,201401,1.0,0.0,1.0,6.0
1,2136680,201403,1.0,0.0,3.0,12.0
2,2136680,201501,2.0,0.0,6.0,14.0
3,2136680,201503,2.0,0.0,8.0,17.0
4,2136680,201601,2.0,0.0,9.0,21.0
...,...,...,...,...,...,...
1040,2231780,202303,0.0,0.0,1.0,7.0
1041,2231781,202303,0.0,0.0,0.0,8.0
1042,2231877,202303,0.0,0.0,5.0,3.0
1043,2232058,202303,0.0,0.0,1.0,7.0


In [23]:
temp2

nivel_nota,id_estudiante,periodo,per_ant_nivel_menos_1,per_ant_nivel_1,per_ant_nivel_2,per_ant_nivel_3
0,2136680,201401,1.0,0.0,1.0,6.0
1,2136680,201403,1.0,0.0,3.0,12.0
2,2136680,201501,2.0,0.0,6.0,14.0
3,2136680,201503,2.0,0.0,8.0,17.0
4,2136680,201601,2.0,0.0,9.0,21.0
...,...,...,...,...,...,...
1040,2231780,202303,0.0,0.0,1.0,7.0
1041,2231781,202303,0.0,0.0,0.0,8.0
1042,2231877,202303,0.0,0.0,5.0,3.0
1043,2232058,202303,0.0,0.0,1.0,7.0


In [24]:
# ahora hacemos el join con DATOS
DATOS = DATOS.merge(temp2, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.fillna(0)


In [53]:
DATOS

,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,periodo_orden,...,historia,quimica,lenguaje,geografia,idioma,interdisciplinar,per_ant_nivel_menos_1,per_ant_nivel_1,per_ant_nivel_2,per_ant_nivel_3
0,0,2136689,201803,201803.0,0,1989-12-14,MAS,LICEO REMINGTON,MEDELLIN,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2146669,201801,201801.0,2018-12-01 00:00:00,1996-08-28,FEM,COLEGIO REYES CATOLICOS,CALI,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2146707,201801,201801.0,0,1997-03-09,FEM,COLEGIO SAN FRANCISCO DE ASIS - CALI,CALI,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2146709,201801,201801.0,2019-04-06 00:00:00,1997-04-20,MAS,LICEO ANGLO DEL VALLE,CALI,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2146710,201801,201801.0,0,1996-08-11,FEM,COLEGIO MIXTO SAN VICENTE,CALI,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,827,2236079,202303,202303.0,0,2001-03-18,MAS,CENTRO DE EDUCACION EN TECNOLOGIA CENTEC,CALI,12,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0,0.0,0.0,0.0
828,828,2236229,202303,202303.0,0,2001-02-25,FEM,COLEGIO MONTERREY,BOGOTÁ D.C.,12,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0,0.0,0.0,0.0
829,829,2236496,202303,202303.0,0,2001-12-19,MAS,COLEGIO BOSTON DE TEUSAQUILLO,BOGOTÁ D.C.,12,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0,0.0,0.0,0.0
830,830,2236497,202303,0.0,0,2006-05-22,FEM,LICEO LA ENSEÑANZA,PALMIRA,12,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0,0.0,0.0,0.0


### **3.3 cantidad de cursos extracurriculares tomados antes**

Ahora calculamos cuántos cursos extracurriculares había tomado antes del periodo actual

In [25]:
query = """
    SELECT   distinct e.codigo id_estudiante, pr.orden periodo_orden, a.tipo
    from estudiantesactividades ea, actividades a, estudiante e, periodosregulares pr
    where a.nombre = ea.nombre_actividad
    and e.identificacion = ea.identificacion
    and str_to_date(substring(fecha_asistencia, 1,8),'%d/%m/%y') between pr.fecha_inicio and pr.fecha_final
"""
temp = pd.read_sql(query, con=motor)

print(temp)

     id_estudiante  periodo_orden     tipo
0          2147700             17  DEPORTE
1          2157148             17  DEPORTE
2          2157148             18  DEPORTE
3          2167326             18  DEPORTE
4          2167326             14  DEPORTE
..             ...            ...      ...
375        2220174             21  DEPORTE
376        2221159             18     OTRO
377        2221159             19     OTRO
378        2225960             19  DEPORTE
379        2225960             20     OTRO

[380 rows x 3 columns]


In [26]:
# seleccionarmos solo id_estudiante, periodo y periodo_orden de DATOS
temp2 = DATOS[['id_estudiante','periodo','periodo_orden']]

# hacemos left join con temp
temp2 = temp2.merge(temp, on=['id_estudiante'], how='left')

# dejamos solo las filas donde periodo_orden_x > periodo_orden_y pues queremos mirar al periodo cuantas
# materias tuvo sin nota, cuantas tuvo con nivel 1, cuantas con nivel 2 y cuantas con nivel 3 de periodos
# anteriores
temp2 = temp2[temp2['periodo_orden_x'] > temp2['periodo_orden_y']]
temp2

# # ahora agrupamos para saber si tuvo al menos una asistencia por periodo.
temp2 = temp2.groupby(['id_estudiante','periodo','tipo'])['periodo_orden_x'].count().reset_index()
temp2 = temp2.rename(columns={'periodo_orden_x':'cuantos'})
temp2

# pivoteamos
temp2 = temp2.pivot(index=['id_estudiante','periodo'], columns='tipo', values='cuantos').reset_index()
temp2 = temp2.fillna(0)
temp2

# renombramos y llenamos con ceros
temp2 = temp2.rename(columns={'CULTURA':'per_ant_cultura', 'DEPORTE':'per_ante_deporte','OTRO':'per_ant_otro'})
temp2

tipo,id_estudiante,periodo,per_ant_cultura,per_ante_deporte,per_ant_otro
0,2186473,202001,0.0,1.0,0.0
1,2186473,202003,0.0,2.0,0.0
2,2186473,202101,0.0,3.0,0.0
3,2186473,202103,0.0,3.0,0.0
4,2186473,202201,0.0,4.0,0.0
...,...,...,...,...,...
173,2231780,202303,0.0,1.0,1.0
174,2231781,202303,0.0,1.0,1.0
175,2231877,202303,0.0,1.0,1.0
176,2232058,202303,0.0,0.0,1.0


In [27]:
# ahora hacemos el join con DATOS
DATOS = DATOS.merge(temp2, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.fillna(0)
DATOS

,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,periodo_orden,...,geografia,idioma,interdisciplinar,per_ant_nivel_menos_1,per_ant_nivel_1,per_ant_nivel_2,per_ant_nivel_3,per_ant_cultura,per_ante_deporte,per_ant_otro
0,0,2136680,201303,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2136680,201401,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,2,...,0.0,0.0,0.0,1.0,0.0,1.0,6.0,0.0,0.0,0.0
2,2,2136680,201403,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,3,...,0.0,0.0,0.0,1.0,0.0,3.0,12.0,0.0,0.0,0.0
3,3,2136680,201501,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,4,...,0.0,0.0,0.0,2.0,0.0,6.0,14.0,0.0,0.0,0.0
4,4,2136680,201503,201303,2017-12-01 00:00:00,1994-09-07 00:00:00,FEM,COLEGIO AMERICANO - CALI,CALI,5,...,0.0,0.0,0.0,2.0,0.0,8.0,17.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,1338,2231877,202303,202301,0,2006-02-05 00:00:00,MAS,INSTITUCION EDUCATIVA PRIMAVERA,BOLIVAR,21,...,50.0,57.0,-100.0,0.0,0.0,5.0,3.0,0.0,1.0,1.0
1339,1339,2232058,202301,202301,0,2005-12-01 00:00:00,MAS,OTRO,CALI,20,...,74.0,74.0,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1340,1340,2232058,202303,202301,0,2005-12-01 00:00:00,MAS,OTRO,CALI,21,...,74.0,74.0,-100.0,0.0,0.0,1.0,7.0,0.0,0.0,1.0
1341,1341,2232126,202301,202301,0,2005-01-10 00:00:00,MAS,COLEGIO MAYOR SAN FRANCISCO DE ASIS,CALI,20,...,55.0,55.0,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
DATOS.to_excel('DATOS2.xlsx')

### **3.4 qué edad tenía el estudiante cuando inicio el semestre?**

Calculamos la edad que tenía el estudiante cuando tomó el semestre


In [35]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)
DATOS = pd.read_excel('DATOS2.xlsx')

In [32]:
DATOS.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id_estudiante', 'periodo',
       'periodo_inicial', 'fecha_grado', 'fecha_nacimiento', 'genero',
       'colegio', 'ciudad_colegio', 'periodo_orden', 'periodo_fi',
       'periodo_ff', 'LABEL', 'icfes_antiguo', 'biologia', 'matematica',
       'filosofia', 'fisica', 'historia', 'quimica', 'lenguaje', 'geografia',
       'idioma', 'interdisciplinar', 'per_ant_nivel_menos_1',
       'per_ant_nivel_1', 'per_ant_nivel_2', 'per_ant_nivel_3',
       'per_ant_cultura', 'per_ante_deporte', 'per_ant_otro',
       'periodo_fecha_inicio'],
      dtype='object')

In [31]:
DATOS.dtypes

Unnamed: 0.1                      int64
Unnamed: 0                        int64
id_estudiante                     int64
periodo                           int64
periodo_inicial                   int64
fecha_grado                      object
fecha_nacimiento                 object
genero                           object
colegio                          object
ciudad_colegio                   object
periodo_orden                     int64
periodo_fi               datetime64[ns]
periodo_ff               datetime64[ns]
LABEL                             int64
icfes_antiguo                     int64
biologia                        float64
matematica                      float64
filosofia                       float64
fisica                          float64
historia                        float64
quimica                         float64
lenguaje                        float64
geografia                       float64
idioma                          float64
interdisciplinar                float64


In [37]:
query = """
    SELECT   periodo, fecha_inicio periodo_fecha_inicio
    FROM periodosregulares
"""
temp = pd.read_sql(query, con=motor)
DATOS = DATOS.merge(temp, on='periodo', how='inner')

# Convertir la fecha de inicio del periodo a un objeto datetime
DATOS['periodo_fi'] = pd.to_datetime( DATOS['periodo_fi'])
DATOS['fecha_nacimiento'] = pd.to_datetime( DATOS['fecha_nacimiento'])

# calculamos la edad
DATOS['estudiante_edad'] = round((DATOS['periodo_fi'] - DATOS['fecha_nacimiento']).dt.days/365,1)
DATOS

,Unnamed: 0.1,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,...,per_ant_nivel_menos_1,per_ant_nivel_1,per_ant_nivel_2,per_ant_nivel_3,per_ant_cultura,per_ante_deporte,per_ant_otro,periodo_fecha_inicio_x,periodo_fecha_inicio_y,estudiante_edad
0,0,0,2136680,201303,201303,2017-12-01 00:00:00,1994-09-07,FEM,COLEGIO AMERICANO - CALI,CALI,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,18.8
1,8,8,2136681,201303,201303,0,1996-01-16,MAS,COLEGIO SENDEROS,JAMUNDI,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,17.5
2,9,9,2136682,201303,201303,2017-12-01 00:00:00,1995-09-28,MAS,CENTRO EDUCATIVO COMFANDI MIRAFLORES,CALI,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,17.8
3,17,17,2136683,201303,201303,2017-12-01 00:00:00,1995-07-13,MAS,COLEGIO NACIONAL MIXTO SAN LUIS GONZAGA,TÚQUERRES,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,18.0
4,25,25,2136684,201303,201303,0,1994-08-03,FEM,COLEGIO SAN FRANCISCO,TULUA,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,18.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,1333,1333,2231780,202303,202301,0,2005-02-05,FEM,0,0,...,0,0,1,7,0,1,1,2023-07-01,2023-07-01,18.4
1339,1335,1335,2231781,202303,202301,0,2006-01-17,FEM,COLEGIO LOS ANGELES DEL NORTE,CALI,...,0,0,0,8,0,1,1,2023-07-01,2023-07-01,17.5
1340,1338,1338,2231877,202303,202301,0,2006-02-05,MAS,INSTITUCION EDUCATIVA PRIMAVERA,BOLIVAR,...,0,0,5,3,0,1,1,2023-07-01,2023-07-01,17.4
1341,1340,1340,2232058,202303,202301,0,2005-12-01,MAS,OTRO,CALI,...,0,0,1,7,0,0,1,2023-07-01,2023-07-01,17.6


In [38]:
DATOS['estudiante_edad'].value_counts()

estudiante_edad
18.0    40
19.0    38
18.5    36
19.7    34
20.2    34
        ..
28.6     1
46.0     1
28.2     1
28.9     1
34.4     1
Name: count, Length: 120, dtype: int64

In [39]:
DATOS.head()

,Unnamed: 0.1,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,...,per_ant_nivel_menos_1,per_ant_nivel_1,per_ant_nivel_2,per_ant_nivel_3,per_ant_cultura,per_ante_deporte,per_ant_otro,periodo_fecha_inicio_x,periodo_fecha_inicio_y,estudiante_edad
0,0,0,2136680,201303,201303,2017-12-01 00:00:00,1994-09-07,FEM,COLEGIO AMERICANO - CALI,CALI,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,18.8
1,8,8,2136681,201303,201303,0,1996-01-16,MAS,COLEGIO SENDEROS,JAMUNDI,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,17.5
2,9,9,2136682,201303,201303,2017-12-01 00:00:00,1995-09-28,MAS,CENTRO EDUCATIVO COMFANDI MIRAFLORES,CALI,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,17.8
3,17,17,2136683,201303,201303,2017-12-01 00:00:00,1995-07-13,MAS,COLEGIO NACIONAL MIXTO SAN LUIS GONZAGA,TÚQUERRES,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,18.0
4,25,25,2136684,201303,201303,0,1994-08-03,FEM,COLEGIO SAN FRANCISCO,TULUA,...,0,0,0,0,0,0,0,2013-07-01,2013-07-01,18.9


### **3.5 por depto, nota min, max, avg y count que sacó en  asignaturas vistas en periodos anteriores anteriores**

Lo que hago, es agrupar por cod_dep y luego calculo las min, max, avg y count() sobre nota definitiva (el count solo cuenta cantidad de filas, es decir cuántas asignaturas vio en semestres anteriores de asignaturas de ese depto)

Esto aplicaría también para:
* genero docente masculino
* edad docente

Así lo hice en javeriana y funcionó..  tal vez valga la pena intentarlo?

In [40]:
query = """
SELECT ea.id_estudiante, ea.periodo, ea2.cod_dep, min(ea2.definitiva) min_definitiva, 
                                                  max(ea2.definitiva) max_definitiva, 
                                                  avg(ea2.definitiva) avg_definitiva,
                                                  count(ea2.definitiva) cnt_definitiva
FROM estudianteasignatura ea, estudianteasignatura ea2
WHERE ea.id_estudiante = ea2.id_estudiante
AND   ea.periodo > ea2.periodo
GROUP BY ea.id_estudiante, ea.periodo, ea2.cod_dep
"""
temp = pd.read_sql(query, con=motor)
temp

# pivoteamos MIN
temp1 = temp.pivot(index=['id_estudiante','periodo'], columns='cod_dep', values='min_definitiva').reset_index()
temp1

# si está nulo es porque no la había visto en ese momento.  -5 significará eso
temp1 = temp1.fillna(-5)
temp1

# colocamos un prefijo
prefijo = 'asig_antes_min_def_'
temp1.columns = [f'{prefijo}{col}' for col in temp1.columns]
temp1

temp1 = temp1.rename(columns={'asig_antes_min_def_id_estudiante':'id_estudiante', 'asig_antes_min_def_periodo':'periodo'})
temp1

# pivoteamos MAX
temp2 = temp.pivot(index=['id_estudiante','periodo'], columns='cod_dep', values='max_definitiva').reset_index()
temp2

# si está nulo es porque no la había visto en ese momento.  -5 significará eso
temp2 = temp2.fillna(-5)
temp2

# colocamos un prefijo
prefijo = 'asig_antes_max_def_'
temp2.columns = [f'{prefijo}{col}' for col in temp2.columns]
temp2

temp2 = temp2.rename(columns={'asig_antes_max_def_id_estudiante':'id_estudiante', 'asig_antes_max_def_periodo':'periodo'})
temp2

# pivoteamos AVG
temp3 = temp.pivot(index=['id_estudiante','periodo'], columns='cod_dep', values='avg_definitiva').reset_index()
temp3

# si está nulo es porque no la había visto en ese momento.  -5 significará eso
temp3 = temp3.fillna(-5)
temp3

# colocamos un prefijo
prefijo = 'asig_antes_avg_def_'
temp3.columns = [f'{prefijo}{col}' for col in temp3.columns]
temp3

temp3 = temp3.rename(columns={'asig_antes_avg_def_id_estudiante':'id_estudiante', 'asig_antes_avg_def_periodo':'periodo'})
temp3

# pivoteamos CNT
temp4 = temp.pivot(index=['id_estudiante','periodo'], columns='cod_dep', values='cnt_definitiva').reset_index()
temp4

# si está nulo es porque no la había visto en ese momento.  -5 significará eso
temp4 = temp4.fillna(-5)
temp4

# colocamos un prefijo
prefijo = 'asig_antes_cnt_def_'
temp4.columns = [f'{prefijo}{col}' for col in temp4.columns]
temp4

temp4 = temp4.rename(columns={'asig_antes_cnt_def_id_estudiante':'id_estudiante', 'asig_antes_cnt_def_periodo':'periodo'})
temp4

# ahora pegamos temp1 a temp4 en DATA (primero con temp)

DATOS = DATOS.merge(temp1, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.merge(temp2, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.merge(temp3, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.merge(temp4, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.fillna(-5)






In [41]:
DATOS = DATOS.fillna(-10)
DATOS.head()

,Unnamed: 0.1,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,...,asig_antes_cnt_def_DIRADM,asig_antes_cnt_def_DIRCIENB,asig_antes_cnt_def_DIRCOMU,asig_antes_cnt_def_DIRHUMA,asig_antes_cnt_def_DIRING,asig_antes_cnt_def_DIS,asig_antes_cnt_def_DME,asig_antes_cnt_def_DOYS,asig_antes_cnt_def_IDI,asig_antes_cnt_def_MER
0,0,0,2136680,201303,201303,2017-12-01 00:00:00,1994-09-07,FEM,COLEGIO AMERICANO - CALI,CALI,...,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
1,8,8,2136681,201303,201303,0,1996-01-16,MAS,COLEGIO SENDEROS,JAMUNDI,...,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
2,9,9,2136682,201303,201303,2017-12-01 00:00:00,1995-09-28,MAS,CENTRO EDUCATIVO COMFANDI MIRAFLORES,CALI,...,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
3,17,17,2136683,201303,201303,2017-12-01 00:00:00,1995-07-13,MAS,COLEGIO NACIONAL MIXTO SAN LUIS GONZAGA,TÚQUERRES,...,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
4,25,25,2136684,201303,201303,0,1994-08-03,FEM,COLEGIO SAN FRANCISCO,TULUA,...,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0


### **3.6 Experiencia Docente semestres anteriores**

La idea es determinar qué tipo de profesores tuvo el estudiante en semestres anteriores.
La experiencia docente puede medirse indirectamente como la cantidad de veces que había dictado la misma asignatura al momento de dictarla en d (DATOS), y la cantidad de veces que había dictado CUALQUIER asignatura antes de dictar la asignatura en d (DATOS)

la idea es obtener una tabla que tenga la siguiente informacion
docente, periodo, asignatura, cantidad_de_veces_dictada_antes

y tambien:
docente, periodo, asignatura, cantidad_de_veces_dicto_cualquier_asignatura_antes

iniciemos con el primer caso.


In [42]:
# 3.6 EXPERIENCIA DOCENTE EN LA MISMA ASIGNATURA
# definimos la experiencia docente de un docente en una asignatura y periodo, 
# como el número de grupos que había dictado en períodos anteriores de esa misma asignatura
# ENTONCES:  dado un estudiante y un periodo, 
# se obtiene la experiencia docente de todos los docentes de las asignaturas que ese estudiante tomó en períodos anteriores

query = """
    select id_estudiante, periodo, cod_dep,  avg(experiencia) avg_experiencia
    from (
        select ed.id_estudiante, ed.periodo, ed.cod_asig, ed.id_docente, 
               ed.cod_dep, count(*) experiencia
        from ea_docentes ed, docentes_grupos dg
        where dg.id_docente = ed.id_docente
        and   dg.periodo < ed.periodo
        and   dg.cod_asig = ed.cod_asig
	group by ed.id_estudiante, ed.periodo, ed.cod_asig, ed.id_docente, ed.cod_dep
) v
group by id_estudiante, periodo, cod_dep    
"""
temp = pd.read_sql(query, con=motor)
temp = temp.pivot(index=['id_estudiante','periodo'], columns='cod_dep', values='avg_experiencia').reset_index()
# si está nulo es porque no la había visto en ese momento.  -10 significará eso
temp = temp.fillna(-10)
prefijo = 'exp_doc_ant_avg_'
temp.columns = [f'{prefijo}{col}' for col in temp.columns]
temp = temp.rename(columns={'exp_doc_ant_avg_id_estudiante':'id_estudiante', 'exp_doc_ant_avg_periodo':'periodo'})
DATOS = DATOS.merge(temp, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.fillna(-10)


In [43]:
DATOS.head()

,Unnamed: 0.1,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,...,exp_doc_ant_avg_DES,exp_doc_ant_avg_DIRADM,exp_doc_ant_avg_DIRCIENB,exp_doc_ant_avg_DIRCOMU,exp_doc_ant_avg_DIRHUMA,exp_doc_ant_avg_DIRING,exp_doc_ant_avg_DME,exp_doc_ant_avg_DOYS,exp_doc_ant_avg_IDI,exp_doc_ant_avg_MER
0,0,0,2136680,201303,201303,2017-12-01 00:00:00,1994-09-07,FEM,COLEGIO AMERICANO - CALI,CALI,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,8,8,2136681,201303,201303,0,1996-01-16,MAS,COLEGIO SENDEROS,JAMUNDI,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,9,9,2136682,201303,201303,2017-12-01 00:00:00,1995-09-28,MAS,CENTRO EDUCATIVO COMFANDI MIRAFLORES,CALI,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,17,17,2136683,201303,201303,2017-12-01 00:00:00,1995-07-13,MAS,COLEGIO NACIONAL MIXTO SAN LUIS GONZAGA,TÚQUERRES,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,25,25,2136684,201303,201303,0,1994-08-03,FEM,COLEGIO SAN FRANCISCO,TULUA,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


### **3.7 Experiencia Docente semestre actual**

Hacemos el mismo procesamiento anterior pero solo tomando en cuenta las asignaturas actuales

In [44]:
query = """
    select id_estudiante, periodo, cod_dep,  avg(experiencia) avg_experiencia
    from (
        select ed.id_estudiante, ed.periodo, ed.cod_asig, 
               ed.id_docente, ed.cod_dep, count(*) experiencia
        from ea_docentes ed, docentes_grupos dg
        where dg.id_docente = ed.id_docente
        and   dg.periodo = ed.periodo
        and   dg.cod_asig = ed.cod_asig
	group by ed.id_estudiante, ed.periodo, ed.cod_asig, ed.id_docente, ed.cod_dep
) v
group by id_estudiante, periodo, cod_dep    
"""
temp = pd.read_sql(query, con=motor)
temp = temp.pivot(index=['id_estudiante','periodo'], columns='cod_dep', values='avg_experiencia').reset_index()

# si está nulo es porque no la esta tomando este  momento.  -10 significará eso
temp = temp.fillna(-10)
prefijo = 'exp_doc_act_avg_'
temp.columns = [f'{prefijo}{col}' for col in temp.columns]
temp = temp.rename(columns={'exp_doc_act_avg_id_estudiante':'id_estudiante', 'exp_doc_act_avg_periodo':'periodo'})
DATOS = DATOS.merge(temp, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.fillna(-10)

In [45]:
pd.set_option('display.max_columns', None)

DATOS.head()

,Unnamed: 0.1,Unnamed: 0,id_estudiante,periodo,periodo_inicial,fecha_grado,fecha_nacimiento,genero,colegio,ciudad_colegio,periodo_orden,periodo_fi,periodo_ff,LABEL,icfes_antiguo,biologia,matematica,filosofia,fisica,historia,quimica,lenguaje,geografia,idioma,interdisciplinar,per_ant_nivel_menos_1,per_ant_nivel_1,per_ant_nivel_2,per_ant_nivel_3,per_ant_cultura,per_ante_deporte,per_ant_otro,periodo_fecha_inicio_x,periodo_fecha_inicio_y,estudiante_edad,asig_antes_min_def_CDH,asig_antes_min_def_CEM,asig_antes_min_def_DAF,asig_antes_min_def_DCA,asig_antes_min_def_DCC,asig_antes_min_def_DCE,asig_antes_min_def_DCJ,asig_antes_min_def_DCS,asig_antes_min_def_DDA,asig_antes_min_def_DDH,asig_antes_min_def_DDL,asig_antes_min_def_DDM,asig_antes_min_def_DES,asig_antes_min_def_DIRADM,asig_antes_min_def_DIRCIENB,asig_antes_min_def_DIRCOMU,asig_antes_min_def_DIRHUMA,asig_antes_min_def_DIRING,asig_antes_min_def_DIS,asig_antes_min_def_DME,asig_antes_min_def_DOYS,asig_antes_min_def_IDI,asig_antes_min_def_MER,asig_antes_max_def_CDH,asig_antes_max_def_CEM,asig_antes_max_def_DAF,asig_antes_max_def_DCA,asig_antes_max_def_DCC,asig_antes_max_def_DCE,asig_antes_max_def_DCJ,asig_antes_max_def_DCS,asig_antes_max_def_DDA,asig_antes_max_def_DDH,asig_antes_max_def_DDL,asig_antes_max_def_DDM,asig_antes_max_def_DES,asig_antes_max_def_DIRADM,asig_antes_max_def_DIRCIENB,asig_antes_max_def_DIRCOMU,asig_antes_max_def_DIRHUMA,asig_antes_max_def_DIRING,asig_antes_max_def_DIS,asig_antes_max_def_DME,asig_antes_max_def_DOYS,asig_antes_max_def_IDI,asig_antes_max_def_MER,asig_antes_avg_def_CDH,asig_antes_avg_def_CEM,asig_antes_avg_def_DAF,asig_antes_avg_def_DCA,asig_antes_avg_def_DCC,asig_antes_avg_def_DCE,asig_antes_avg_def_DCJ,asig_antes_avg_def_DCS,asig_antes_avg_def_DDA,asig_antes_avg_def_DDH,asig_antes_avg_def_DDL,asig_antes_avg_def_DDM,asig_antes_avg_def_DES,asig_antes_avg_def_DIRADM,asig_antes_avg_def_DIRCIENB,asig_antes_avg_def_DIRCOMU,asig_antes_avg_def_DIRHUMA,asig_antes_avg_def_DIRING,asig_antes_avg_def_DIS,asig_antes_avg_def_DME,asig_antes_avg_def_DOYS,asig_antes_avg_def_IDI,asig_antes_avg_def_MER,asig_antes_cnt_def_CDH,asig_antes_cnt_def_CEM,asig_antes_cnt_def_DAF,asig_antes_cnt_def_DCA,asig_antes_cnt_def_DCC,asig_antes_cnt_def_DCE,asig_antes_cnt_def_DCJ,asig_antes_cnt_def_DCS,asig_antes_cnt_def_DDA,asig_antes_cnt_def_DDH,asig_antes_cnt_def_DDL,asig_antes_cnt_def_DDM,asig_antes_cnt_def_DES,asig_antes_cnt_def_DIRADM,asig_antes_cnt_def_DIRCIENB,asig_antes_cnt_def_DIRCOMU,asig_antes_cnt_def_DIRHUMA,asig_antes_cnt_def_DIRING,asig_antes_cnt_def_DIS,asig_antes_cnt_def_DME,asig_antes_cnt_def_DOYS,asig_antes_cnt_def_IDI,asig_antes_cnt_def_MER,exp_doc_ant_avg_DAF,exp_doc_ant_avg_DCE,exp_doc_ant_avg_DCJ,exp_doc_ant_avg_DCS,exp_doc_ant_avg_DDA,exp_doc_ant_avg_DDH,exp_doc_ant_avg_DDL,exp_doc_ant_avg_DDM,exp_doc_ant_avg_DES,exp_doc_ant_avg_DIRADM,exp_doc_ant_avg_DIRCIENB,exp_doc_ant_avg_DIRCOMU,exp_doc_ant_avg_DIRHUMA,exp_doc_ant_avg_DIRING,exp_doc_ant_avg_DME,exp_doc_ant_avg_DOYS,exp_doc_ant_avg_IDI,exp_doc_ant_avg_MER,exp_doc_act_avg_nan,exp_doc_act_avg_CDH,exp_doc_act_avg_CEM,exp_doc_act_avg_DAF,exp_doc_act_avg_DCA,exp_doc_act_avg_DCC,exp_doc_act_avg_DCE,exp_doc_act_avg_DCJ,exp_doc_act_avg_DCS,exp_doc_act_avg_DDA,exp_doc_act_avg_DDH,exp_doc_act_avg_DDL,exp_doc_act_avg_DDM,exp_doc_act_avg_DES,exp_doc_act_avg_DIRADM,exp_doc_act_avg_DIRCIENB,exp_doc_act_avg_DIRCOMU,exp_doc_act_avg_DIRHUMA,exp_doc_act_avg_DIRING,exp_doc_act_avg_DIS,exp_doc_act_avg_DME,exp_doc_act_avg_DOYS,exp_doc_act_avg_IDI,exp_doc_act_avg_MER
0,0,0,2136680,201303,201303,2017-12-01 00:00:00,1994-09-07,FEM,COLEGIO AMERICANO - CALI,CALI,1,2013-07-01,2013-12-31,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,2013-07-01,2013-07-01,18.8,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0

### **3.8  Promedios obtenidos en periodos anteriores**

dado un d en DATOS, miramos el orden del periodo.   
Ahora miramos los promedios que tiene calculados en períodos anteriores y obtenemos la distancia con el periodo actual.
Y creamos una tabla que tenga:
id_estudiante, periodo, prom_hace_1, prom_hace_2, ...  

In [46]:
query = """
    select em1.id_estudiante, em1.periodo, em2.prom_ponderado_sem promsem, pr1.orden - pr2.orden distancia
    from estudiantesmatriculas em1, periodosregulares pr1, estudiantesmatriculas em2, periodosregulares pr2
    where pr1.periodo = em1.periodo
    and   em2.id_estudiante = em1.id_estudiante
    and   pr2.periodo = em2.periodo
    and   pr1.orden > pr2.orden
"""
temp = pd.read_sql(query, con=motor)

temp = temp.pivot(index=['id_estudiante','periodo'], columns='distancia', values='promsem').reset_index()

prefijo = 'promsem_hace_'
temp.columns = [f'{prefijo}{col}' for col in temp.columns]
temp = temp.rename(columns={'promsem_hace_id_estudiante':'id_estudiante', 'promsem_hace_periodo':'periodo'})
temp = temp.fillna(-5)
temp['periodo'] = temp['periodo'].astype('int64')
DATOS = DATOS.merge(temp, on=['id_estudiante','periodo'], how='left')
DATOS = DATOS.fillna(-5)



,id_estudiante,periodo,promsem_hace_1,promsem_hace_2,promsem_hace_3,promsem_hace_4,promsem_hace_5,promsem_hace_6,promsem_hace_7,promsem_hace_8,promsem_hace_9,promsem_hace_10,promsem_hace_11,promsem_hace_12
0,2126591,201401,4.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
1,2126591,201403,3.6,4.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
2,2126591,201501,4.3,3.6,4.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
3,2126591,201503,4.1,4.3,3.6,4.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
4,2126591,201601,4.3,4.1,4.3,3.6,4.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2205868,202103,-5.0,4.7,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
1019,2205924,202101,3.1,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
1020,2205924,202103,3.1,3.1,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0
1021,2206167,202101,4.6,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0,-5.0


In [49]:
DATOS.to_excel('DATOS3.xlsx')

In [50]:
DATOS.columns


Index(['Unnamed: 0.1', 'Unnamed: 0', 'id_estudiante', 'periodo',
       'periodo_inicial', 'fecha_grado', 'fecha_nacimiento', 'genero',
       'colegio', 'ciudad_colegio',
       ...
       'promsem_hace_3', 'promsem_hace_4', 'promsem_hace_5', 'promsem_hace_6',
       'promsem_hace_7', 'promsem_hace_8', 'promsem_hace_9', 'promsem_hace_10',
       'promsem_hace_11', 'promsem_hace_12'],
      dtype='object', length=181)

In [51]:
DATOS['LABEL'].value_counts()

LABEL
 0    1030
 1     179
-1     134
Name: count, dtype: int64

In [53]:

# Listar las columnas sin nombre
columnas_sin_nombre = [col for col in DATOS.columns if 'Unnamed:' in col]
columnas_sin_nombre
DATOS = DATOS.drop(columnas_sin_nombre, axis=1)


In [54]:
DATOS.columns

Index(['id_estudiante', 'periodo', 'periodo_inicial', 'fecha_grado',
       'fecha_nacimiento', 'genero', 'colegio', 'ciudad_colegio',
       'periodo_orden', 'periodo_fi',
       ...
       'promsem_hace_3', 'promsem_hace_4', 'promsem_hace_5', 'promsem_hace_6',
       'promsem_hace_7', 'promsem_hace_8', 'promsem_hace_9', 'promsem_hace_10',
       'promsem_hace_11', 'promsem_hace_12'],
      dtype='object', length=179)

In [55]:
columnas_a_borrar = ['id_estudiante','periodo','periodo_inicial','fecha_grado', 
                     'fecha_nacimiento','periodo_orden', 'periodo_fi','periodo_ff']
DATOS = DATOS.drop(columnas_a_borrar, axis=1)

In [56]:
DATOS.columns

Index(['genero', 'colegio', 'ciudad_colegio', 'LABEL', 'icfes_antiguo',
       'biologia', 'matematica', 'filosofia', 'fisica', 'historia',
       ...
       'promsem_hace_3', 'promsem_hace_4', 'promsem_hace_5', 'promsem_hace_6',
       'promsem_hace_7', 'promsem_hace_8', 'promsem_hace_9', 'promsem_hace_10',
       'promsem_hace_11', 'promsem_hace_12'],
      dtype='object', length=171)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold